In [60]:
import asyncio
import csv
import json
import os
import re
from typing import Optional

import nest_asyncio
import pandas as pd
import tqdm
from g4f.client import AsyncClient
from googletrans import Translator
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

## Refactor and save tasks dict

In [61]:
CLEAN_TAGS_R = re.compile("<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")


def remove_html_tags(text: str) -> str:
    return re.sub(CLEAN_TAGS_R, "", text)

In [62]:
def batch_list(strings: list[str], batch_size: int) -> list[list[str]]:
    return [strings[i : i + batch_size] for i in range(0, len(strings), batch_size)]


async def translate_text(text: str, src: str = "ru", dest: str = "en") -> str:
    translator = Translator()
    translation = await translator.translate(text, src=src, dest=dest)
    return translation.text


async def translate_texts_inner(
    texts: list[str], src: str = "ru", dest: str = "en"
) -> list[str]:
    translator = Translator()
    if len(texts) > 1:
        return [res.text for res in await translator.translate(texts, src=src, dest=dest)]
    res = await translator.translate(texts[0], src=src, dest=dest)
    return [res.text]


async def translate_texts(texts: list[str]) -> list[str]:
    res = await tqdm_asyncio.gather(
        *[translate_texts_inner(text_list) for text_list in batch_list(texts, 5)]
    )

    ans = []
    for r in res:
        ans.extend(r)
    return ans

In [63]:
def save_dict(tasks_dict: dict, path: str):
    with open(path, "w", encoding="utf8") as f:
        json.dump(tasks_dict, f, ensure_ascii=False, indent=4)


def build_task_str(task: dict) -> str:
    return remove_html_tags(
        ""
        # f"{task['title']}\n\n"
        # + f"Теги:{task['tags']}\n\n"
        + f"{task['description']}\n\n"
        + f"Формат входных данных:\n{task['inputFormat']}\n\n"
        + f"Формат выходных данных:\n{task['outputFormat']}\n\n"
        # + f"Примеры\n{task['examples']}"
        # + (f"\n\nПримечание:\n{task['remark']}" if task["remark"] != "" else "")
    )


def compile_tasks_dict() -> dict[str, str]:
    df = pd.read_csv("../../data/db_tasks.csv")
    raw_dict = df.fillna("").set_index("spec").to_dict(orient="index")

    return {str(spec): build_task_str(task) for spec, task in raw_dict.items()}


async def save_dicts():
    tasks_dict = compile_tasks_dict()
    save_dict(tasks_dict, "../../data/generated/db_tasks_ru.json")

    specs, values = zip(*tasks_dict.items())
    translated_values = await translate_texts(list(values))
    tasks_dict_en = dict(zip(specs, translated_values))
    save_dict(tasks_dict_en, "../../data/generated/db_tasks_en.json")


# await save_dicts()

## Load tasks dict

In [64]:
def load_dict(path: str) -> dict:
    try:
        with open(path, "r", encoding="utf8") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}


tasks_dict: dict[str, str] = load_dict("../../data/generated/db_tasks_en.json")
len(tasks_dict)

553

## Generate solutions

In [65]:
AVAILABLE_MODELS = [
    # "gpt-3.5-turbo",
    # "gpt-4",
    # "gpt-4o",
    # "gpt-4o-mini",
    # "o1",
    "o1-mini",
    "o3-mini",
    # "gigachat",
    # "meta-ai",
    "llama-2-7b",
    "llama-3-8b",
    "llama-3-70b",
    "llama-3.1-8b",
    "llama-3.1-70b",
    # "llama-3.1-405b",
    "llama-3.2-1b",
    "llama-3.2-3b",
    "llama-3.2-11b",
    # "llama-3.2-90b",
    "llama-3.3-70b",
    "mixtral-8x7b",
    "mixtral-8x22b",
    "mistral-nemo",
    "mixtral-small-24b",
    # "hermes-3",
    "phi-3.5-mini",
    "phi-4",
    # "wizardlm-2-7b",
    # "wizardlm-2-8x22b",
    "gemini-exp",
    "gemini-1.5-flash",
    "gemini-1.5-pro",
    "gemini-2.0",
    # "gemini-2.0-flash",
    # "gemini-2.0-flash-thinking",
    # "gemini-2.0-pro",
    # "claude-3-haiku",
    # "claude-3-sonnet",
    # "claude-3-opus",
    # "claude-3.5-sonnet",
    # "claude-3.7-sonnet",
    # "claude-3.7-sonnet-thinking",
    "reka-core",
    "blackboxai",
    "blackboxai-pro",
    "command-r",
    "command-r-plus",
    "command-r7b",
    "command-a",
    # "qwen-1.5-7b",
    "qwen-2-72b",
    "qwen-2-vl-7b",
    "qwen-2.5-72b",
    "qwen-2.5-coder-32b",
    "qwen-2.5-1m",
    "qwen-2-5-max",
    "qwq-32b",
    "qvq-72b",
    "pi",
    "deepseek-chat",
    "deepseek-v3",
    # "deepseek-r1",
    # "janus-pro-7b",
    # "grok-3",
    # "grok-3-r1",
    # "sonar",
    # "sonar-pro",
    # "sonar-reasoning",
    # "sonar-reasoning-pro",
    # "r1-1776",
    "nemotron-70b",
    # "dbrx-instruct",
    # "glm-4",
    # "yi-34b",
    # "dolphin-2.6",
    # "dolphin-2.9",
    # "airoboros-70b",
    # "lzlv-70b",
    # "minicpm-2.5",
    # "tulu-3-1-8b",
    # "tulu-3-70b",
    # "tulu-3-405b",
    # "olmo-1-7b",
    "olmo-2-13b",
    "olmo-2-32b",
    "olmo-4-synthetic",
    "lfm-40b",
    "evil",
]

AVAILABLE_MODELS = [
    "evil",
    # "llama-3.1-8b",
    "llama-3.2-3b",
    "blackboxai",
    "blackboxai-pro",
    "deepseek-chat",
    "deepseek-v3",
]


def PROMPT(task):
    return f"""Write a Python solution for the following task.
The code should look like it was written by an intermediate student: practical but not overly optimized or perfect. Follow these guidelines:
1. Use not overly long names (e.g., res instead of result or final_output_value).
2. Do not include comments or explanations
3. Avoid using functions, prefer straightforward logic
4. Apply small stylistic deviations, like mixing single/double quotes, occasional redundant logic, inconsistent spacing, etc
5. No error handling
6. Do not print to output anything except answer to the problem without any annotations
 Finally, return just pure python code included in ```python ``` section
Task: \n{task}."""

In [66]:
def extract_python_code(res: str) -> str:
    return (
        re.findall(r"```python(.*?)```", res, re.DOTALL)[0]
        .encode("utf-8", errors="ignore")
        .decode("utf-8")
    )


async def access_llm(
    client: AsyncClient, model: str, prompt: str
) -> tuple[Optional[str], str, str]:
    try:
        response = await client.chat.completions.create(
            model=model, messages=[{"role": "user", "content": prompt}], web_search=False
        )
    except Exception as e:
        return None, model, str(e)
    res = ""
    try:
        res = response.choices[0].message.content
        return extract_python_code(response.choices[0].message.content), model, ""
    except Exception as e2:
        return None, model, str(e2) + (f"\n{res}" if len(res) > 0 else "")

In [67]:
OUTPUT_COLUMNS = ["task", "code", "model"]
ERR_COLUMNS = ["task", "model", "error"]


def setup_output(path: str, columns: list[str] = OUTPUT_COLUMNS):
    with open(path, "w", newline="") as f:
        csv.DictWriter(f, fieldnames=columns).writeheader()


def append_output(path: str, data: dict, columns: list[str] = OUTPUT_COLUMNS):
    with open(path, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=columns)
        writer.writerow(data)

In [68]:
async def generate_solutions(
    task_dict: dict,
    output: str,
    output_err: str,
    prompt=PROMPT,
    models: list[str] = AVAILABLE_MODELS,
    timeout: float = 60.0,
) -> list:
    if not os.path.exists(output):
        setup_output(output, columns=OUTPUT_COLUMNS)
    if not os.path.exists(output_err):
        setup_output(output_err, columns=ERR_COLUMNS)

    solutions = []

    for spec, task in tqdm.tqdm(task_dict.items()):
        client = AsyncClient()

        results = [access_llm(client, m, prompt(task)) for m in models]

        for coro in asyncio.as_completed(results):
            try:
                solution, m, err = await asyncio.wait_for(coro, timeout)
                if solution is None or len(solution) == 0:
                    append_output(
                        output_err,
                        {"task": spec, "model": m, "error": err},
                        columns=ERR_COLUMNS,
                    )
                    continue
                solutions.append((spec, m, solution))
                append_output(
                    output,
                    {"task": spec, "code": solution, "model": m},
                    columns=OUTPUT_COLUMNS,
                )
            except asyncio.TimeoutError:
                continue
            except BaseException:
                continue
        await asyncio.sleep(2)

    return solutions

In [69]:
pass_dict = {}
task_specs = list(tasks_dict.keys())
idx = task_specs.index("5bab2114-725a-4548-aa0e-c6b7296898d5")
pass_dict = {k: tasks_dict[k] for k in task_specs[idx + 1 :]}

In [ ]:
# pass_dict = {
#     k: tasks_dict[k]
#     for k in [
#         "c0df7d49-26f5-451c-b44a-1e0bca60bca5",
#         "4e5b21c0-e86f-4eac-82b6-1a0d00ae4199",
#     ]
# }
# pass_dict  = tasks_dict

# solutions = await generate_solutions(pass_dict, "test.csv", "test_log.csv", timeout=5)
solutions = []

  1%|          | 1/85 [00:06<08:38,  6.17s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001AB83917380>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x000001AB83544770>, 41724.015)])']
connector: <aiohttp.connector.TCPConnector object at 0x000001ABFFF7B710>
 18%|█▊        | 15/85 [03:22<15:52, 13.60s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001AB836765D0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x000001AB838B6870>, 41920.468)])']
connector: <aiohttp.connector.TCPConnector object at 0x000001AB83676D50>
 32%|███▏      | 27/85 [04:57<07:38,  7.91s/it]Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001ABFFEF9EE0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x000001AB83966210>, 42015.812)])']
connector: <aiohttp.connector.TCPCon

In [72]:
print(f"Solutions = {len(solutions)}")
print(f"Possible solutions = {len(pass_dict) * len(AVAILABLE_MODELS)}")

Solutions = 483
Possible solutions = 510


In [ ]:
res_df = pd.read_csv("results.csv")
len(res_df)

3177